In [ ]:
import random
import data
import os
import parameters as p
import tensorflow as tf
import importlib

In [ ]:
importlib.reload(p)
importlib.reload(data)

In [ ]:
random.seed()
classes = data.retrieve_class_names()
output_dim = p.GX * p.GY * (5 + len(classes))
pages = data.Retriever().retrieve(classes)

In [ ]:
files = os.listdir(p.PNG_PATH)
generator = data.Generator(pages, output_dim, classes)
ds = tf.data.Dataset.from_generator(
    generator.generator,
    args=[files],
    output_types=(tf.int32, tf.float32),
    output_shapes=([p.BATCH_SIZE, p.Y, p.X, 3], [p.BATCH_SIZE, output_dim]))

In [ ]:
from keras import backend as K

def get_loss(y_actual,y_pred):

    y_actual_reshaped = K.reshape(y_actual, [p.BATCH_SIZE * p.GX * p.GY, -1])
    y_pred_reshaped = K.reshape(y_pred, [p.BATCH_SIZE * p.GX * p.GY, -1])

    main_probs_act = y_actual_reshaped[..., :1]
    main_probs_pred = y_pred_reshaped[..., :1]
    bb_probs_act = y_actual_reshaped[..., 1:5]
    bb_probs_pred =  y_pred_reshaped[..., 1:5]
    classes_probs_act = y_actual_reshaped[..., 5:]
    classes_probs_pred =  y_pred_reshaped[..., 5:]

    loss = K.sum(K.binary_crossentropy(main_probs_act, main_probs_pred, from_logits=True))
    loss += K.sum(K.sum(K.abs(bb_probs_act - bb_probs_pred), 1) * main_probs_act)
    loss += K.sum(K.sum(K.abs(classes_probs_act - classes_probs_pred), 1) * main_probs_act)

    return loss


In [ ]:
vgg = tf.keras.applications.vgg19.VGG19(
    include_top=False, weights='imagenet',
    input_shape=(p.Y,p.X,3), pooling='avg'
)

model = tf.keras.Sequential(vgg.layers)
model.add(tf.keras.layers.Dense(output_dim, activation='relu'))

In [ ]:
importlib.reload(p)
importlib.reload(data)
model.compile(optimizer='adam',
              loss=get_loss,
              metrics=tf.keras.metrics.MeanAbsoluteError(name="mean_absolute_error", dtype=None))
a = model. fit(ds, epochs=1)

In [ ]:
aa = generator.generator(files).__next__()
aaa = model.predict(aa[0])
get_loss(aa[1].astype('float32'), aaa)

a = aa[0][0]
import viewer as v
v.show_prediction(aa[1][0], aa[0][0])
